# Sprint
## 機械学習フロー
丁寧な検証が行える状態にした上で、他者の解法を参考に汎化性能の高いモデル作りを進めます。

# 【問題1】
## クロスバリデーション
事前学習期間では検証データをはじめに分割しておき、それに対して指標値を計算することで検証を行っていました。（ホールドアウト法）しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション（交差検証） を行います。分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割のためにscikit-learnにはKFoldクラスが用意されています。

事前学習期間の課題で作成したベースラインモデルに対してKFoldクラスによるクロスバリデーションを行うコードを作成し実行してください。

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# データセット読み込み
credits_train = pd.read_csv('application_train.csv')
credits_test = pd.read_csv('application_test.csv')

In [3]:
X = credits_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].values
y = credits_train['TARGET']

In [4]:
def scale_and_fill(X, scaler):
    # 標準化
    X_norm = scaler.transform(X)

    # 欠損値確認
    nan_num = np.isnan(X_norm).sum()

    # 穴埋め用乱数生成 - arr_replace
    rands = np.random.normal(0, 1, nan_num)
    arr_replace = np.zeros(X_norm.shape)
    np.place(arr_replace, np.isnan(X_norm), rands)

    # 欠損値穴埋め
    X_norm_filled = np.where(np.isnan(X_norm), arr_replace, X_norm)
    return X_norm_filled

In [10]:
# KFold検証
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

kf = KFold(n_splits=5)
scaler = StandardScaler()
rforest = RandomForestClassifier()
print(f'split: {kf.get_n_splits(X)}')
auc_arr = np.zeros(kf.get_n_splits(X))

for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # スケーリングとか
    scaler.fit(X_train)
    X_train_new = scale_and_fill(X_train, scaler)
    # 学習・予測
    rforest.fit(X_train_new, y_train)
    # バリデーション
    X_test_new = scale_and_fill(X_test, scaler)
    y_test_pred = rforest.predict_proba(X_test_new)[:, 1]
    auc_score = roc_auc_score(y_test, y_test_pred)
    auc_arr[i] = auc_score
    print(f'{i+1}. AUC - test: {auc_score}')

print(f'AUC - test/average: {np.mean(auc_arr)}')

split: 5
1. AUC - test: 0.6455288079408439
2. AUC - test: 0.6517000510935217
3. AUC - test: 0.6497391949973208
4. AUC - test: 0.6522689109152051
5. AUC - test: 0.6506231475455044
AUC - test/average: 0.6499720224984792


# 【問題2】
## グリッドサーチ
これまで分類器のパラメータには触れず、デフォルトの設定を使用していました。パラメータの詳細は今後のSprintで学んでいくことになります。機械学習の前提として、パラメータは状況に応じて最適なものを選ぶ必要があります。最適なパラメータを探していくことを パラメータチューニング と呼びます。パラメータチューニングをある程度自動化する単純な方法としては グリッドサーチ があります。

scikit-learnのGridSearchCVを使い、グリッドサーチを行うコードを作成してください。そして、ベースラインモデルに対して何らかしらのパラメータチューニングを行なってください。どのパラメータをチューニングするかは、使用した手法の公式ドキュメントを参考にしてください。

[sklearn.model_selection.GridSearchCV — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

GridSearchCVクラスには引数としてモデル、探索範囲、さらにクロスバリデーションを何分割で行うかを与えます。クロスバリデーションの機能も含まれているため、これを使用する場合はKFoldクラスを利用する必要はありません。

In [13]:
# 最終チェック用
from sklearn.model_selection import train_test_split

X2_train, X2_test, y2_train, y2_test = train_test_split(X, y, stratify=y)

scaler.fit(X2_train)
X2_train_new = scale_and_fill(X2_train, scaler)

In [14]:
# グリッドサーチ
from sklearn.model_selection import GridSearchCV

rforest2 = RandomForestClassifier()
params = {
    'n_estimators': [10, 50, 100, 500],
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [1, 2, 5]
}
gscv = GridSearchCV(rforest2, param_grid=params, cv=3, verbose=1)
gscv.fit(X2_train_new, y2_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 47.9min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [15]:
# 最適パラメータ確認
print(f'best score: {gscv.best_score_}')
print(f'best params: {gscv.best_params_}')

best score: 0.9180602944355566
best params: {'criterion': 'gini', 'min_samples_split': 5, 'n_estimators': 500}


In [19]:
# 最適パラメータでバリデーションを行う
from sklearn.metrics import confusion_matrix

X2_test_new = scale_and_fill(X2_test, scaler)
best = gscv.best_estimator_
y2_test_pred = best.predict_proba(X2_test_new)[:, 1]

print(f'AUC: {roc_auc_score(y2_test, y2_test_pred)}')
print(best)

AUC: 0.6680807596824182
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [35]:
# testデータに対する推測を行う
def out_pred_csv(df, X, model, filename='out_pred.csv'):
    """
    信用情報分析コンペ向けcsv排出関数
    df: ID取得用DataFrame
    X: 処理済み特徴量（ID含まない） ndarray
    model: 予測モデル
    """
    ids = df['SK_ID_CURR'].values
    y_pred = model.predict_proba(X)[:, 1]
    submit_arr = np.concatenate((ids.reshape(-1, 1), y_pred.reshape(-1, 1)), axis=1)
    submit_df = pd.DataFrame(submit_arr, columns=['SK_ID_CURR', 'TARGET'])
    submit_df['SK_ID_CURR'] = submit_df['SK_ID_CURR'].astype(np.int32)
    display(submit_df)
    submit_df.info()
    submit_df.to_csv(filename, index=False)

    return True

In [31]:
X_test_submit = scale_and_fill(credits_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].values, scaler)

In [36]:
print(out_pred_csv(credits_test, X_test_submit, best, filename='sprint1-2.csv'))

,SK_ID_CURR,TARGET
0,100001,0.016143
1,100005,0.121458
2,100013,0.004667
3,100028,0.044424
4,100038,0.132554
...,...,...
48739,456221,0.054848
48740,456222,0.019690
48741,456223,0.029643
48742,456224,0.024417


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SK_ID_CURR  48744 non-null  int32  
 1   TARGET      48744 non-null  float64
dtypes: float64(1), int32(1)
memory usage: 571.3 KB
True


# 【問題3】
## Kaggle Notebooksからの調査
KaggleのNotebooksから様々なアイデアを見つけ出して、列挙してください。

# 【問題4】
## 高い汎化性能のモデル作成
問題3で見つけたアイデアと、独自のアイデアを組み合わせ高い汎化性能のモデル作りを進めてください。

その過程として、何を行うことで、クロスバリデーションの結果がどの程度変化したかを表にまとめてください。

# 【問題5】
## 最終的なモデルの選定
最終的にこれは良いというモデルを選び、推定した結果をKaggleに提出してスコアを確認してください。どういったアイデアを取り入れ、どの程度のスコアになったかを記載してください。